In [1]:
import numpy as np
import pandas as pd
import glob
import datetime as dt
import csv
import re

### Washington D.C

In [2]:
DC_311_2019 = '/Users/julia/Box/Data_311/DMV_311/DMV_nonEM/DC/311_Washington_DC_2019.csv'
DC_311_2020 = '/Users/julia/Box/Data_311/DMV_311/DMV_nonEM/DC/311_Washington_DC_2020.csv'
DC_311_2019_df = pd.read_csv(DC_311_2019)
DC_311_2020_df = pd.read_csv(DC_311_2020)

/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3071: DtypeWarning: Columns (12,14,16) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,
/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3071: DtypeWarning: Columns (29) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


1. DC 2019

In [3]:
DC_311_2019_df.head()

,X,Y,OBJECTID,SERVICECODE,SERVICECODEDESCRIPTION,SERVICETYPECODEDESCRIPTION,ORGANIZATIONACRONYM,SERVICECALLCOUNT,ADDDATE,RESOLUTIONDATE,...,XCOORD,YCOORD,LATITUDE,LONGITUDE,CITY,STATE,ZIPCODE,MARADDRESSREPOSITORYID,WARD,DETAILS
0,-76.930018,38.904475,278741,S0031,Bulk Collection,SWMA- Solid Waste Management Admistration,DPW,1,2019/07/18 00:57:15+00,2019/08/05 19:42:09+00,...,406070.32,137403.48,38.904467,-76.930016,WASHINGTON,DC,20019.0,15976.0,7.0,NaN
1,-77.034081,38.941341,278742,S05SL,Streetlight Repair Investigation,Transportation Operations Administration,DDOT,1,2019/07/18 01:00:00+00,2019/07/21 03:03:10+00,...,397045.59,141494.19,38.941334,-77.034079,WASHINGTON,DC,20011.0,259155.0,4.0,NaN
2,-77.034326,38.915358,278743,S05SL,Streetlight Repair Investigation,Transportation Operations Administration,DDOT,1,2019/07/18 01:58:52+00,2019/07/19 07:23:15+00,...,397023.24,138609.83,38.915350,-77.034324,WASHINGTON,DC,20009.0,240017.0,2.0,NaN
3,-76.942364,38.898615,278744,S0336,Out of State Parking Violation (ROSA),PEMA- Parking Enforcement Management Administr...,DPW,1,2019/07/18 01:59:10+00,2019/07/18 06:39:36+00,...,404999.88,136752.30,38.898608,-76.942362,WASHINGTON,DC,20019.0,25746.0,7.0,NaN
4,-76.978173,38.888846,278745,S0316,Recycling Cart Delivery,SWMA- Solid Waste Management Admistration,DPW,1,2019/07/18 01:01:44+00,2019/08/19 11:09:05+00,...,401893.88,135666.52,38.888839,-76.978170,WASHINGTON,DC,20003.0,55558.0,6.0,NaN


In [4]:
DC_311_2019_df['opened_date'] = pd.to_datetime(DC_311_2019_df['ADDDATE'], format='%Y/%m/%d %H:%M:%S')
print(DC_311_2019_df.columns)
DC_311_2019_df = DC_311_2019_df.loc[ DC_311_2019_df['opened_date'].dt.year == 2019]
print('#attributes: ' + str(len(DC_311_2019_df.columns) -1))
DC_311_2019_df.drop_duplicates(subset=['OBJECTID'], inplace = True)
min_date = min(DC_311_2019_df['opened_date'])
max_date = max(DC_311_2019_df['opened_date'])
print("#311 reports between " + str(min_date) + " and " + str(max_date) + ": " + str(DC_311_2019_df.shape[0]))

Index(['X', 'Y', 'OBJECTID', 'SERVICECODE', 'SERVICECODEDESCRIPTION',
       'SERVICETYPECODEDESCRIPTION', 'ORGANIZATIONACRONYM', 'SERVICECALLCOUNT',
       'ADDDATE', 'RESOLUTIONDATE', 'SERVICEDUEDATE', 'SERVICEORDERDATE',
       'INSPECTIONFLAG', 'INSPECTIONDATE', 'INSPECTORNAME',
       'SERVICEORDERSTATUS', 'STATUS_CODE', 'SERVICEREQUESTID', 'PRIORITY',
       'STREETADDRESS', 'XCOORD', 'YCOORD', 'LATITUDE', 'LONGITUDE', 'CITY',
       'STATE', 'ZIPCODE', 'MARADDRESSREPOSITORYID', 'WARD', 'DETAILS',
       'opened_date'],
      dtype='object')
#attributes: 30
#311 reports between 2019-01-01 05:15:41+00:00 and 2019-12-31 23:59:53+00:00: 364928


2. DC 2020

In [5]:
DC_311_2020_df.head()

,X,Y,OBJECTID,SERVICECODE,SERVICECODEDESCRIPTION,SERVICETYPECODEDESCRIPTION,ORGANIZATIONACRONYM,SERVICECALLCOUNT,ADDDATE,RESOLUTIONDATE,...,XCOORD,YCOORD,LATITUDE,LONGITUDE,CITY,STATE,ZIPCODE,MARADDRESSREPOSITORYID,WARD,DETAILS
0,-77.052255,38.901881,1,S05SL,Streetlight Repair Investigation,Transportation Operations Administration,DDOT,1,2020/01/01 05:00:38+00,2020/01/04 08:35:04+00,...,395467.53,137114.59,38.901874,-77.052253,WASHINGTON,DC,20037.0,273983.0,2.0,NaN
1,-76.995047,38.853735,2,S0311,Rodent Inspection and Treatment,DOH- Department Of Health,DOH,1,2020/01/01 05:02:57+00,2020/01/01 05:12:17+00,...,400430.11,131768.69,38.853728,-76.995045,WASHINGTON,DC,20032.0,301360.0,8.0,NaN
2,-77.024210,38.913582,3,S0441,Trash Collection - Missed,SWMA- Solid Waste Management Admistration,DPW,1,2020/01/01 05:20:59+00,2020/01/02 13:08:05+00,...,397900.54,138412.45,38.913575,-77.024208,WASHINGTON,DC,20001.0,239705.0,6.0,NaN
3,-77.025296,38.905826,4,S0441,Trash Collection - Missed,SWMA- Solid Waste Management Admistration,DPW,1,2020/01/01 06:34:21+00,2020/01/02 13:05:05+00,...,397806.12,137551.45,38.905818,-77.025294,WASHINGTON,DC,20001.0,239755.0,2.0,NaN
4,-77.014662,38.881105,5,EMNPV,Emergency No-Parking Verification,PEMA- Parking Enforcement Management Administr...,DPW,1,2020/01/01 09:07:20+00,2020/01/01 09:18:13+00,...,398728.03,134807.00,38.881097,-77.014660,WASHINGTON,DC,20024.0,277827.0,6.0,NaN


In [6]:
DC_311_2020_df['opened_date'] = pd.to_datetime(DC_311_2020_df['ADDDATE'], format='%Y/%m/%d %H:%M:%S')
DC_311_2020_df = DC_311_2020_df.loc[ DC_311_2020_df['opened_date'].dt.year == 2020]
print(DC_311_2020_df.columns)
print('#attributes: ' + str(len(DC_311_2020_df.columns)-1))
DC_311_2020_df.drop_duplicates(subset=['OBJECTID'], inplace = True)
min_date = min(DC_311_2020_df['opened_date'])
max_date = max(DC_311_2020_df['opened_date'])
print("#311 reports between " + str(min_date) + " and " + str(max_date) + ": " + str(DC_311_2020_df.shape[0]))

Index(['X', 'Y', 'OBJECTID', 'SERVICECODE', 'SERVICECODEDESCRIPTION',
       'SERVICETYPECODEDESCRIPTION', 'ORGANIZATIONACRONYM', 'SERVICECALLCOUNT',
       'ADDDATE', 'RESOLUTIONDATE', 'SERVICEDUEDATE', 'SERVICEORDERDATE',
       'INSPECTIONFLAG', 'INSPECTIONDATE', 'INSPECTORNAME',
       'SERVICEORDERSTATUS', 'STATUS_CODE', 'SERVICEREQUESTID', 'PRIORITY',
       'STREETADDRESS', 'XCOORD', 'YCOORD', 'LATITUDE', 'LONGITUDE', 'CITY',
       'STATE', 'ZIPCODE', 'MARADDRESSREPOSITORYID', 'WARD', 'DETAILS',
       'opened_date'],
      dtype='object')
#attributes: 30
#311 reports between 2020-01-01 05:00:38+00:00 and 2020-12-31 23:56:22+00:00: 304718


### Montgomery County

In [7]:
MC_311_all = '/Users/julia/Box/Data_311/DMV_311/DMV_nonEM/Montgomery County/311_Montgomery County_MD_2012_2021.csv'
MC_311_all_df = pd.read_csv(MC_311_all)
MC_311_all_df.drop_duplicates(subset=['SR ID'], inplace = True)
print(MC_311_all_df.columns)
print('#attributes: ' + str(len(MC_311_all_df.columns)))

Index(['SR ID', 'Opened', 'Closed', 'Status', 'Department', 'Area', 'Sub-Area',
       'Attached Solution (Topic)', 'Attached Solution SLA Days', 'City',
       'State', 'Zip Code', 'Source', 'Election District',
       'Maryland State District', 'Congressional District',
       'Congressional Member', 'Council District', 'Council Member Name',
       'Changed Date', '# of Days Open', 'Within SLA Windows', 'SLA Yes',
       'SLA No'],
      dtype='object')
#attributes: 24


In [8]:
MC_311_all_df['opened_date'] = pd.to_datetime(MC_311_all_df['Opened'], format='%m/%d/%Y %I:%M:%S %p')
MC_311_all_df.head()

,SR ID,Opened,Closed,Status,Department,Area,Sub-Area,Attached Solution (Topic),Attached Solution SLA Days,City,...,Congressional District,Congressional Member,Council District,Council Member Name,Changed Date,# of Days Open,Within SLA Windows,SLA Yes,SLA No,opened_date
0,1303668945,10/11/2016 12:33:37 PM,10/13/2016 02:24:33 PM,Closed,DEP,Solid Waste,Scrap Metal Request,Scrap Metal Pick-Up Request,5.0,SILVER SPRING,...,8.0,"CHRISTOPHER VAN HOLLEN,JR -Dem",5.0,Tom Hucker,10/13/2016 06:24:34 PM,2,Yes,1,0,2016-10-11 12:33:37
1,1305658856,11/15/2016 02:25:38 PM,11/18/2016 07:57:00 AM,Closed,DEP,Solid Waste,Scrap Metal Request,Scrap Metal Pick-Up Request,5.0,ROCKVILLE,...,8.0,"CHRISTOPHER VAN HOLLEN,JR -Dem",4.0,Nancy Navarro,11/18/2016 12:57:00 PM,3,Yes,1,0,2016-11-15 14:25:38
2,1310137174,02/06/2017 08:33:04 AM,02/06/2017 08:35:08 AM,Closed,DPS,Residential Construction,Residential Inspections,Name and Telephone Number of DPS Residential B...,1.0,NaN,...,NaN,NaN,NaN,NaN,02/06/2017 01:35:10 PM,0,Yes,1,0,2017-02-06 08:33:04
3,1310137190,02/06/2017 08:40:43 AM,02/06/2017 08:41:17 AM,Closed,FIN,Payroll,NaN,Employee - Questions on the W-2 Form,1.0,NaN,...,NaN,NaN,NaN,NaN,02/06/2017 01:41:18 PM,0,Yes,1,0,2017-02-06 08:40:43
4,1377072702,01/15/2019 09:54:44 AM,01/18/2019 12:48:24 PM,Closed,DOT,Highway Services,Snow Removal,Report Unplowed or Missed Street,1.0,SILVER SPRING,...,8.0,"CHRISTOPHER VAN HOLLEN,JR -Dem",4.0,Nancy Navarro,01/18/2019 05:48:27 PM,3,No,0,1,2019-01-15 09:54:44


1. Montgomery County 2019

In [9]:
MC_311_2019 = MC_311_all_df.loc[ MC_311_all_df['opened_date'].dt.year == 2019]
MC_311_2019.reset_index(drop = True, inplace = True)
# MC_311_2019.to_csv('/Users/julia/Box/Data_311/DMV_311/DMV_nonEM/Montgomery County/311_Montgomery County_MD_2019.csv')

In [10]:
print(MC_311_2019.columns)
print('#attributes: ' + str(len(MC_311_2019.columns)-1))
min_date = min(MC_311_2019['opened_date'])
max_date = max(MC_311_2019['opened_date'])
print("#311 reports between " + str(min_date) + " and " + str(max_date) + ": " + str(MC_311_2019.shape[0]))

Index(['SR ID', 'Opened', 'Closed', 'Status', 'Department', 'Area', 'Sub-Area',
       'Attached Solution (Topic)', 'Attached Solution SLA Days', 'City',
       'State', 'Zip Code', 'Source', 'Election District',
       'Maryland State District', 'Congressional District',
       'Congressional Member', 'Council District', 'Council Member Name',
       'Changed Date', '# of Days Open', 'Within SLA Windows', 'SLA Yes',
       'SLA No', 'opened_date'],
      dtype='object')
#attributes: 24
#311 reports between 2019-01-01 00:22:32 and 2019-12-31 22:51:04: 555509


2. Montgomery County 2020

In [11]:
MC_311_2020 = MC_311_all_df.loc[ MC_311_all_df['opened_date'].dt.year == 2020]
MC_311_2020.reset_index(drop = True, inplace = True)
# MC_311_2020.to_csv('/Users/julia/Box/Data_311/DMV_311/DMV_nonEM/Montgomery County/311_Montgomery County_MD_2020.csv')

In [12]:
print(MC_311_2020.columns)
print('#attributes: ' + str(len(MC_311_2020.columns)-1))
min_date = min(MC_311_2020['opened_date'])
max_date = max(MC_311_2020['opened_date'])
print("#311 reports between " + str(min_date) + " and " + str(max_date) + ": " + str(MC_311_2020.shape[0]))

Index(['SR ID', 'Opened', 'Closed', 'Status', 'Department', 'Area', 'Sub-Area',
       'Attached Solution (Topic)', 'Attached Solution SLA Days', 'City',
       'State', 'Zip Code', 'Source', 'Election District',
       'Maryland State District', 'Congressional District',
       'Congressional Member', 'Council District', 'Council Member Name',
       'Changed Date', '# of Days Open', 'Within SLA Windows', 'SLA Yes',
       'SLA No', 'opened_date'],
      dtype='object')
#attributes: 24
#311 reports between 2020-01-01 02:08:54 and 2020-12-31 23:19:24: 576486


In [13]:
MC_311_2020['Status'].unique()

array(['Closed', 'In Progress'], dtype=object)

### Prince George's County

In [14]:
PG_311_all = '/Users/julia/Box/Data_311/DMV_311/DMV_nonEM/Prince George County/311_Prince George County_MD_20210310.csv'
PG_311_all = pd.read_csv(PG_311_all)
PG_311_all.drop_duplicates(subset=['Service Request #'], inplace = True)
print(PG_311_all.columns)
print('#attributes: ' + str(len(PG_311_all.columns)))

Index(['Service Request #', 'Date Request Opened', 'Request Status', 'SLA',
       'Service Group', 'Agency Responsible', 'Request Name', 'Street Address',
       'Latitude', 'Longitude', 'Location', 'TNI Area Description'],
      dtype='object')
#attributes: 12


In [15]:
PG_311_all['opened_date'] = pd.to_datetime(PG_311_all['Date Request Opened'], format='%m/%d/%Y %I:%M:%S %p')
PG_311_all.head()

,Service Request #,Date Request Opened,Request Status,SLA,Service Group,Agency Responsible,Request Name,Street Address,Latitude,Longitude,Location,TNI Area Description,opened_date
0,18-00103211,09/22/2018 10:45:44 AM,OPEN,OVERDUE,Administration - Sheriff,Sheriff,Status of Open Warrants,NaN,NaN,NaN,NaN,County Address,2018-09-22 10:45:44
1,18-00103242,09/23/2018 03:25:01 PM,CLOSED,CLOSED,3-1-1 Call Center (Ocr),Community Relations (Ocr),Agency Feedback (Positive Experience/Negative ...,4325 SPRING FOAL CT,NaN,NaN,NaN,County Address,2018-09-23 15:25:01
2,18-00103255,09/23/2018 09:46:43 PM,OPEN,CLOSED,Multi Family,DPIE,ENF-Rental Issues Apartment Unit,7304 RIGGS RD,NaN,NaN,NaN,County Address,2018-09-23 21:46:43
3,18-00103239,09/23/2018 01:56:45 PM,OPEN,OVERDUE,Public Safety Communications,Homeland Security,PSC Audio Recording Request,NaN,NaN,NaN,NaN,County Address,2018-09-23 13:56:45
4,18-00103251,09/23/2018 07:09:02 PM,OPEN,OVERDUE,Public Safety Communications,Homeland Security,PSC Audio Recording Request,NaN,NaN,NaN,NaN,County Address,2018-09-23 19:09:02


1. Prince George's County 2019

In [16]:
PG_311_2019 = PG_311_all.loc[ PG_311_all['opened_date'].dt.year == 2019]
PG_311_2019.reset_index(drop = True, inplace = True)
# PG_311_2019.to_csv('/Users/julia/Box/Data_311/DMV_311/DMV_nonEM/Prince George County/311_Prince George County_MD_2019.csv') 

PG_311_2020 = PG_311_all.loc[ PG_311_all['opened_date'].dt.year == 2020]
PG_311_2020.reset_index(drop = True, inplace = True)
# PG_311_2020.to_csv('/Users/julia/Box/Data_311/DMV_311/DMV_nonEM/Prince George County/311_Prince George County_MD_2020.csv') 


In [17]:
print(PG_311_2019.columns)
print('#attributes: ' + str(len(PG_311_2019.columns) -1))
min_date = min(PG_311_2019['opened_date'])
max_date = max(PG_311_2019['opened_date'])
print("#311 reports between " + str(min_date) + " and " + str(max_date) + ": " + str(PG_311_2019.shape[0]))

Index(['Service Request #', 'Date Request Opened', 'Request Status', 'SLA',
       'Service Group', 'Agency Responsible', 'Request Name', 'Street Address',
       'Latitude', 'Longitude', 'Location', 'TNI Area Description',
       'opened_date'],
      dtype='object')
#attributes: 12
#311 reports between 2019-01-01 01:21:16 and 2019-12-31 17:15:41: 57553


2. Prince George's County 2020

In [18]:
print(PG_311_2020.columns)
print('#attributes: ' + str(len(PG_311_2020.columns) -1))
min_date = min(PG_311_2020['opened_date'])
max_date = max(PG_311_2020['opened_date'])
print("#311 reports between " + str(min_date) + " and " + str(max_date) + ": " + str(PG_311_2020.shape[0]))

Index(['Service Request #', 'Date Request Opened', 'Request Status', 'SLA',
       'Service Group', 'Agency Responsible', 'Request Name', 'Street Address',
       'Latitude', 'Longitude', 'Location', 'TNI Area Description',
       'opened_date'],
      dtype='object')
#attributes: 12
#311 reports between 2020-01-01 08:08:42 and 2020-12-31 21:57:12: 47145


In [19]:
PG_311_2020['Request Status'].unique()

array(['CLOSED', 'OPEN'], dtype=object)

In [20]:
PG_311_2020['SLA'].unique()

array(['CLOSED', 'OPEN', 'OVERDUE'], dtype=object)

In [21]:
PG_311_2020['Service Group'].unique()

array(['Police Department', 'Special Services Division',
       'Collections-WMD', 'Zoning', 'Traffic Operations Division',
       'Multi Family', 'Single Family (Code Enforcement)',
       'Construction Division', 'Traffic Safety Division',
       'Rva - General', 'Field Activities - Amd',
       'Animal Management Division - A M D', 'Building and Construction',
       'Health', 'Public Safety Communications',
       'Abandoned Vehicle Unit A V U', 'Stormdrain Maintenance Division',
       'Office of Information Technology (Oit)',
       'Food Protection and Policy Program', 'Fire and Rescue Operations',
       'Revenue Authority', 'Administrative Services Division - A S D',
       'Community Relations (Ocr)',
       'Sustainable Initiatives Division - SID', 'Permits',
       'Child Adult and Family Services', '3-1-1 Call Center (Ocr)',
       'Fleet Management Division', 'Office of Highway Maintenance',
       'Office of Transportation', 'Site Road',
       'Environmental Engineering

In [22]:
PG_311_2020['Agency Responsible'].unique()

array(['Police Department', 'Department of Public Works & Transportation',
       'Dept. of The Environment-DOE', 'DPIE', 'Revenue Authority',
       'Health', 'Homeland Security',
       'Office of Information Technology (Oit)',
       'Environmental Health and Disease Control (Hlt)', 'Fire/Ems',
       'Community Relations (Ocr)', 'Social Services',
       'Office of Central Services (Ocs)', 'Board of Elections (Boe)',
       'Board of License (Blc)', "County Executive's Office",
       'Public Safety', 'Office of Finance', 'Sheriff', 'Training Group'],
      dtype=object)

In [23]:
print(PG_311_2020['Request Name'].nunique())
print(PG_311_2019['Request Name'].nunique())

173
177


In [24]:
PG_311_2020['TNI Area Description'].unique()

array(['County Address', 'Kentland / Palmer Park', 'Woodlawn - Lanham',
       'Forestville', 'Bladensburg / East Riverdale', 'Silver Hill',
       'Suitland / Coral Hills', 'Oxon Hill / Glassmanor', 'Langley Park',
       'Hillcrest Heights / Marlow Heights'], dtype=object)

### Arlington County

In [25]:
arlington_311 = '/Users/julia/Box/Data_311/DMV_311/DMV_nonEM/Arlington County/MyArlington_Arlington_VA_2020_2021.txt'
arlington_311_csv = '/Users/julia/Box/Data_311/DMV_311/DMV_nonEM/Arlington County/MyArlington_Arlington_VA_2020_2021.csv'

In [26]:
with open(arlington_311) as txt_file:
    arlington_311_cols = txt_file.readline().rstrip().split('|')
    lines = txt_file.readlines()
    arlington_311_df = pd.DataFrame(columns=arlington_311_cols)
    txt_file.close()  

In [27]:
for line in lines:
    line = line.rstrip()
    line = re.sub('\s+\|\s+', ' ', line)
    record = line.split('|')
    if len(record) < len(arlington_311_cols):
        nan_ls = [np.nan] *(len(arlington_311_cols) - len(record)) 
        record.extend(nan_ls)
    record_series = pd.Series(record, index = arlington_311_cols)
    arlington_311_df =  arlington_311_df.append(record_series, ignore_index=True)
arlington_311_df.to_csv(arlington_311_csv)
arlington_311_df.head()

,service_request_key,service_request_id,name,description,status,longitude,latitude,address,created_at,updated_at,permission,source_system
0,12,7272245,Facilities Damaged,Please control the water flow. Too much,COMPLETED,-77.11682500,38.87067800,"Lubber Run Park,Arlington, VA 22203",2020-01-01 12:51:59,2020-01-08 12:16:19,This is visible to Everyone,PublicStuff
1,28,7272780,Trail Maintenance,"""Sorry this is a weird one but its worth a go....",SUBMITTED,-77.12064024,38.90418015,"2400 N Wakefield Ct, Arlington, VA 22207, USA",2020-01-01 16:46:35,2020-01-01 16:46:35,This is visible to Everyone,PublicStuff
2,44,7273620,Streetlight Damaged/Out,,COMPLETED,-77.13011250,38.87678140,"858 N Jacksonville St, Arlington, VA 22205, USA",2020-01-02 07:18:01,2020-01-02 08:50:30,This is visible to Everyone,PublicStuff
3,60,7274198,Street Surface Damaged,In the intersection with S. Wayne St. - part o...,COMPLETED,-77.08359069,38.87152199,"2220 2nd St S, Arlington, VA 22204, USA",2020-01-02 09:38:22,2020-02-12 08:38:07,This is visible to Everyone,PublicStuff
4,76,7274767,ITS Specialist,No comm,COMPLETED,-77.07080400,38.89824900,"Lee Hwy & North Lynn Street, Arlington, VA, USA",2020-01-02 10:44:27,2020-01-02 10:44:55,This is visible to Everyone,PublicStuff


In [28]:
arlington_311_df['opened_date'] = pd.to_datetime(arlington_311_df['created_at'], format='%Y-%m-%d %H:%M:%S')
arlington_311_df['updated_date'] = pd.to_datetime(arlington_311_df['updated_at'], format='%Y-%m-%d %H:%M:%S')


* Service requests submitted over the past 6 months, so 2019's data is not available

In [29]:
arlington_311_2019 = arlington_311_df.loc[ arlington_311_df['opened_date'].dt.year == 2019]
arlington_311_2019.reset_index(drop = True, inplace = True)

# arlington_311_2019.to_csv('/Users/julia/Box/Data_311/DMV_311/DMV_nonEM/Arlington County/MyArlington_Arlington_VA_2019.csv') 
arlington_311_2019

,service_request_key,service_request_id,name,description,status,longitude,latitude,address,created_at,updated_at,permission,source_system,opened_date,updated_date


In [30]:
arlington_311_2020 = arlington_311_df.loc[ arlington_311_df['opened_date'].dt.year == 2020]
arlington_311_2020.reset_index(drop = True, inplace = True)
# arlington_311_2020.to_csv('/Users/julia/Box/Data_311/DMV_311/DMV_nonEM/Arlington County/MyArlington_Arlington_VA_2020.csv') 
arlington_311_2020.head()

,service_request_key,service_request_id,name,description,status,longitude,latitude,address,created_at,updated_at,permission,source_system,opened_date,updated_date
0,12,7272245,Facilities Damaged,Please control the water flow. Too much,COMPLETED,-77.11682500,38.87067800,"Lubber Run Park,Arlington, VA 22203",2020-01-01 12:51:59,2020-01-08 12:16:19,This is visible to Everyone,PublicStuff,2020-01-01 12:51:59,2020-01-08 12:16:19
1,28,7272780,Trail Maintenance,"""Sorry this is a weird one but its worth a go....",SUBMITTED,-77.12064024,38.90418015,"2400 N Wakefield Ct, Arlington, VA 22207, USA",2020-01-01 16:46:35,2020-01-01 16:46:35,This is visible to Everyone,PublicStuff,2020-01-01 16:46:35,2020-01-01 16:46:35
2,44,7273620,Streetlight Damaged/Out,,COMPLETED,-77.13011250,38.87678140,"858 N Jacksonville St, Arlington, VA 22205, USA",2020-01-02 07:18:01,2020-01-02 08:50:30,This is visible to Everyone,PublicStuff,2020-01-02 07:18:01,2020-01-02 08:50:30
3,60,7274198,Street Surface Damaged,In the intersection with S. Wayne St. - part o...,COMPLETED,-77.08359069,38.87152199,"2220 2nd St S, Arlington, VA 22204, USA",2020-01-02 09:38:22,2020-02-12 08:38:07,This is visible to Everyone,PublicStuff,2020-01-02 09:38:22,2020-02-12 08:38:07
4,76,7274767,ITS Specialist,No comm,COMPLETED,-77.07080400,38.89824900,"Lee Hwy & North Lynn Street, Arlington, VA, USA",2020-01-02 10:44:27,2020-01-02 10:44:55,This is visible to Everyone,PublicStuff,2020-01-02 10:44:27,2020-01-02 10:44:55


In [31]:
print(arlington_311_2020.columns)
print('#attributes: ' + str(len(arlington_311_2020.columns) -2))
min_date = min(arlington_311_2020['opened_date'])
max_date = max(arlington_311_2020['opened_date'])

print("#reports between " + str(min_date) + " and " + str(max_date) + ": " + str(arlington_311_2020.shape[0]))

Index(['service_request_key', 'service_request_id', 'name', 'description',
       'status', 'longitude', 'latitude', 'address', 'created_at',
       'updated_at', 'permission', 'source_system', 'opened_date',
       'updated_date'],
      dtype='object')
#attributes: 12
#reports between 2020-01-01 09:46:03 and 2020-12-31 14:59:00: 10645


In [32]:
arlington_311_2020['status'].unique()

array(['COMPLETED', 'SUBMITTED', 'IN_PROGRESS', 'RECEIVED', 'OTHER',
       'OPEN', 'CLOSED', 'ON_HOLD'], dtype=object)